In [ ]:
from mysql.connector import connect, Error
from flask import Flask, request, jsonify
from werkzeug.security import check_password_hash

In [ ]:
#Connection
def get_db_connection():
    try:
        connection = connect(
            host="localhost",
            user="root",
            password="password",
            database="Offers2day"
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None


In [ ]:
app = Flask(__name__) #Flask app

In [ ]:
#Test inputs

email = "yassin1234@gmail.com"
password = "Yassin123"

In [ ]:
@app.route('/login', methods=['POST'])#login route
def login():
    #User input from the http request
    data = request.get_json() #received the json object from flutter
    email = data.get('email') #user email or username
    password = data.get('password') #user password
    customer_business = "Customer" if data.get("isCustomer") == "True" else "Business" #This will determine whether the user is a customer or a business

    connection = get_db_connection()

    if connection:
        cursor = connection.cursor(dictionary=True)#data will get returned as a dict not a tuple

        #our query
        #customer
        if customer_business == "Customer":
            query = f"SELECT * FROM Customer C, CustomerPasswords CP WHERE C.email = {email} OR C.username = {email} AND C.userID = CP.userID" 
        #business
        else:
            query = f"SELECT * FROM Businesses B, BusinessPasswords BP WHERE B.businessEmail = {email} OR B.businessName = {email} AND B.businessID = BP.businessID" 
        
        #Execute the query
        cursor.execute(query)
        #Get the first result
        user = cursor.fetchone()#dict

        #check_password_hash(user['password'], password):#check_password_hash is a function that hashes the user password before checking the db
        if user and user['password'] == password:
            cursor.close()
            connection.close()
            return jsonify({'message': 'Login successful'}), 200
        else:
            cursor.close()
            connection.close()
            return jsonify({'message': 'Login failed'}), 401